In [ ]:
# pip install extract-msg
# pip install PyPDF2
#pip install os

In [1]:
from extract_msg import Message
import PyPDF2
import os

ModuleNotFoundError: No module named 'PyPDF2'

In [8]:
msg = Message("test.msg")

In [9]:
# Print email details
print("Subject:", msg.subject)
print("Body:", msg.body)
print("Attachments:", msg.attachments)

Subject: Bill for your Airtel Wi-Fi 08017509808_wifi - March'25 
Body: None
Attachments: [<extract_msg.attachments.attachment.Attachment object at 0x0000023CE2281FD0>, <extract_msg.attachments.attachment.Attachment object at 0x0000023CE27D4C80>]


In [ ]:
# Process attachments
for attachment in msg.attachments:
    if attachment.longFilename.endswith(".pdf"):
        pdf_path = os.path.join("attachments", attachment.longFilename)
        attachment.save(customPath="attachments/")  # Save PDF

        # Read PDF content
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                print(page.extract_text())  # Extract text from each page

In [ ]:
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI
from enum import Enum
from typing import List

In [13]:
msg_text = """Money inbound"""

In [14]:
# --------------------------------------------------------------
# Regular Completion using OpenAI (with drawbacks)
# --------------------------------------------------------------

In [ ]:
client = OpenAI()

In [ ]:
def classify_tickets(ticket_text: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Classify the following customer support ticket into a category."},
            {"role": "user", "content": ticket_text}
        ]
    )
    return response.choices[0].message.content

result = classify_ticket_simple(ticket1)
print(result)

In [16]:
# --------------------------------------------------------------
# Step 2: Patch your LLM with instructor
# --------------------------------------------------------------

In [ ]:
# Instructor makes it easy to get structured data like JSON from LLMs
client = instructor.patch(client)

In [27]:
** Step 3: Define Pydantic data models **

SyntaxError: invalid syntax (2788149541.py, line 1)

In [18]:
class TicketCategory(str, Enum):
    REQUEST_TYPE = "request_type"
    SUB_REQUEST_TYPE = "sub_request_type"
    OTHER = "other"

NameError: name 'Enum' is not defined

In [ ]:
class CustomerSentiment(str, Enum):
    RECEIVED = "received"
    SEND = "send"

In [ ]:
class TicketUrgency(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    PRIORITY = "priority"

In [ ]:
class TicketClassification(BaseModel):
    category: TicketCategory
    urgency: TicketUrgency
    sentiment: CustomerSentiment
    confidence: float = Field(ge=0, le=1, description="Confidence score for the classification")
    key_information: List[str] = Field(description="List of key points extracted from the ticket")
    suggested_action: str = Field(description="Brief suggestion for handling the ticket")

In [ ]:
# --------------------------------------------------------------
# Step 4: Bring everything together in a single function
# --------------------------------------------------------------

def classify_ticket(ticket_text: str) -> TicketClassification:
    response = client.chat.completions.create(
        model="gpt-4o",
        response_model=TicketClassification,
        temperature=0,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": "Analyze the following customer support ticket and extract the requested information."
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response


In [ ]:
result = classify_ticket(msg_text)

In [ ]:
print(result.model_dump_json(indent=2))